### Multiple Tests

In [3]:
import pandas as pd
data = pd.read_csv('AUCs.txt','\t')

In [4]:
print data

                 Unnamed: 0   C4.5  C4.5+m  C4.5+cf  C4.5+m+cf
0            adult (sample)  0.763   0.768    0.771      0.798
1             breast cancer  0.599   0.591    0.590      0.569
2   breast cancer wisconsin  0.954   0.971    0.968      0.967
3                       cmc  0.628   0.661    0.654      0.657
4                ionosphere  0.882   0.888    0.886      0.898
5                      iris  0.936   0.931    0.916      0.931
6           liver disorders  0.661   0.668    0.609      0.685
7               lung cancer  0.583   0.583    0.563      0.625
8              lymphography  0.775   0.838    0.866      0.875
9                  mushroom  1.000   1.000    1.000      1.000
10            primary tumor  0.940   0.962    0.965      0.962
11                    rheum  0.619   0.666    0.614      0.669
12                   voting  0.972   0.981    0.975      0.975
13                     wine  0.957   0.978    0.946      0.970


In [11]:
print data.columns

Index([u'Unnamed: 0', u'C4.5', u'C4.5+m', u'C4.5+cf', u'C4.5+m+cf'], dtype='object')


In [41]:
from itertools import combinations
from scipy import stats
combin =combinations(data.columns[1:], 2)

diff_classificator = []
for i, j in combin:
    print i, j, diff_classificator.append((stats.wilcoxon(data[i],data[j])[0], stats.wilcoxon(data[i],data[j])[1]))

 C4.5 C4.5+m None
C4.5 C4.5+cf None
C4.5 C4.5+m+cf None
C4.5+m C4.5+cf None
C4.5+m C4.5+m+cf None
C4.5+cf C4.5+m+cf None


Сравнение алгоритмов и подсчет p-value без учета множественности гипотез

In [51]:
combin =combinations(data.columns[1:], 2)
diff_model = pd.DataFrame.from_records(diff_classificator)
diff_model.columns = ['statistics', 'pvalue']
diff_model.set_index([[i for i in combin]])

statistics    pvalue
C4.5    C4.5+m            6.5  0.010757
        C4.5+cf          43.0  0.861262
        C4.5+m+cf        11.0  0.015906
C4.5+m  C4.5+cf          17.0  0.046333
        C4.5+m+cf        22.0  0.327826
C4.5+cf C4.5+m+cf        10.0  0.022909

In [52]:
(diff_model.pvalue<0.05).value_counts()

True     4
False    2
Name: pvalue, dtype: int64

С поправкой на множественность гипотез. Метод Холма


In [61]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(diff_model.pvalue, 
                                            alpha = 0.05, 
                                            method = 'holm') 
print reject

[False False False False False False]


In [59]:
diff_model['p_corrected'] = p_corrected
diff_model['reject'] = reject
combin =combinations(data.columns[1:], 2)
diff_model.set_index([[i for i in combin]])

statistics    pvalue  p_corrected reject
C4.5    C4.5+m            6.5  0.010757     0.064543  False
        C4.5+cf          43.0  0.861262     0.861262  False
        C4.5+m+cf        11.0  0.015906     0.079532  False
C4.5+m  C4.5+cf          17.0  0.046333     0.138998  False
        C4.5+m+cf        22.0  0.327826     0.655651  False
C4.5+cf C4.5+m+cf        10.0  0.022909     0.091636  False

С поправкой на множественность гипотез. Метод Бенджамини-Хохберга

In [63]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(diff_model.pvalue, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
diff_model['p_corrected'] = p_corrected
diff_model['reject'] = reject
combin =combinations(data.columns[1:], 2)
diff_model.set_index([[i for i in combin]])

statistics    pvalue  p_corrected reject
C4.5    C4.5+m            6.5  0.010757     0.045818   True
        C4.5+cf          43.0  0.861262     0.861262  False
        C4.5+m+cf        11.0  0.015906     0.045818   True
C4.5+m  C4.5+cf          17.0  0.046333     0.069499  False
        C4.5+m+cf        22.0  0.327826     0.393391  False
C4.5+cf C4.5+m+cf        10.0  0.022909     0.045818   True